In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1564494171557_0010,pyspark,idle,,,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-9>

In [2]:
from collections import defaultdict
#import seaborn as sns
#import matplotlib.pyplot as plt
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,functions as F
from pyspark.sql.types import *

In [3]:

g_appname="nest_spark_get_mongodb"

#Local
#g_input="mongodb://127.0.0.1/backend_production.properties"

#Production
#g_input="mongodb://10.0.1.70:27017/backend_production.properties"

g_input="mongodb://3.210.155.32:27017/backend_production.properties"



In [4]:

my_spark = SparkSession \
     .builder \
     .appName("amazon-personalize-items-dataset") \
     .config("spark.mongodb.input.uri", g_input ) \
     .config("spark.sql.pivotMaxValues", 100000) \
     .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.0')\
     .config('spark.driver.memory', "10g")\
     .config('spark.executor.memory', "8g")\
     .config("spark.executor.cores","4")\
     .getOrCreate()

In [17]:

#pipeline="[{'$match': {'status':'active', 'country': 'CA' }},{ $project : {'beds':1,'price_cents':1} } ]"

#pipeline="[{'$match': {'status':'active', 'country': 'CA' }},{ $project : {'beds':1,'price_cents':1,'city':1,'postal':1,'created_at':1,'status':1} } ]"

#pipeline="[{'$match': {'external_type':'list_hub', 'status':'active','country': 'CA' }},{ $project : {'beds':1,'price_cents':1,'city':1,'postal':1,'created_at':1,'status':1,'country':1} } ]"

pipeline="[{'$match': {'status':'active'}},{ $project : {'beds':1,'price_cents':1,'city':1,'postal':1,'created_at':1,'status':1,'country':1} } ]"



In [18]:
P_SCHEMA=StructType(  (
                StructField('_id',StringType(),True),
                StructField('price_cents',IntegerType(),True),
                StructField('beds',IntegerType(),True),
                StructField('city',StringType(),True),
                StructField('postal',StringType(),True),
                StructField('status',StringType(),True),
                StructField('created_at',TimestampType(),True),
                StructField('country',StringType(),True),
               )
            )


In [22]:
sqlContext.clearCache()

In [23]:

df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("pipeline", pipeline)\
    .schema(P_SCHEMA)\
    .option("inferSchema","false")\
    .load()
    #.limit(5000)


In [24]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- price_cents: integer (nullable = true)
 |-- beds: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- postal: string (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- country: string (nullable = true)

In [25]:
df.registerTempTable("df_tbl")

In [26]:
my_spark.catalog.cacheTable("df_tbl")

In [27]:
import time
def unix_time(val):
    return int(time.mktime(time.strptime(val, '%Y-%m-%d')))

spark.udf.register("unix_time", unix_time)

<function unix_time at 0x7f6b833fb5f0>

In [28]:
df_filter = sqlContext.sql('''select _id as ITEM_ID, 
                                        NVL(price_cents,0) as PRICE, 
                                        NVL(beds,0) as BEDS,
                                        NVL(city,'UNKNOWN') as CITY,
                                        NVL(postal,'UNKNOWN') as POSTAL,
                                        unix_time(substr(created_at,1,10)) as CREATED_AT,
                                        country
                                        from df_tbl
                                        order by 6 asc
                            ''')

In [29]:
df_filter.printSchema()

root
 |-- ITEM_ID: string (nullable = true)
 |-- PRICE: integer (nullable = false)
 |-- BEDS: integer (nullable = false)
 |-- CITY: string (nullable = false)
 |-- POSTAL: string (nullable = false)
 |-- CREATED_AT: string (nullable = true)
 |-- country: string (nullable = true)

In [15]:
#df_filter.count()

781

In [30]:
df_filter.repartition(1) \
        .write.format("com.databricks.spark.csv") \
        .partitionBy('country')\
        .mode("overwrite") \
        .option("header","true")\
        .save("s3://nestready-amazon-personalize/items/")


In [ ]:
##ca/items/part-00000-6c04e1b1-ebfd-4328-bca4-5da4b7c17c5c-c000.csv
##ca/items/part-00000-66a7cb5a-3e05-4e27-8d92-f7e8cf83e557-c000.csv

In [31]:
my_spark.catalog.uncacheTable("df_tbl")